In [19]:
import tensorflow as tf
import os

###Loading The Data

In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ravirajsinh45/real-life-industrial-dataset-of-casting-product")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'real-life-industrial-dataset-of-casting-product' dataset.
Path to dataset files: /kaggle/input/real-life-industrial-dataset-of-casting-product


In [21]:
path_to_dataset = "/root/.cache/kagglehub/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product/versions/2"

os.listdir(path_to_dataset)

['casting_512x512', 'casting_data']

In [22]:
os.listdir(os.path.join(path_to_dataset, "casting_data/casting_data"))

['train', 'test']

###Deviding into the Train and Test

The dataset was already dived into the Train and Test data in separate directories. Therefore, it is easy to use

In [23]:
BASE_DIR = os.path.join(path_to_dataset, "casting_data/casting_data")

TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

In [25]:
IMG_SIZE = (300, 300)
BATCH_SIZE = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    labels = "inferred",
    label_mode = "binary",
    color_mode = "grayscale"
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    labels = "inferred",
    label_mode = "binary",
    color_mode = "grayscale"
    )

Found 6633 files belonging to 2 classes.
Found 715 files belonging to 2 classes.


In [26]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
    .cache()
    .shuffle(SHUFFLE_BUFFER_SIZE)
    .prefetch(PREFETCH_BUFFFER_SIZE)
)

test_dataset_final = (test_dataset
    .cache()
    .prefetch(PREFETCH_BUFFFER_SIZE)
)

###Model Implementation

In [28]:
cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(300, 300, 1)),

    #Normalizing the pixels in the image
    tf.keras.layers.Rescaling(1./255),

    #Concolutional layers
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    #Flatterning Layer
    tf.keras.layers.GlobalAveragePooling2D(),

    #Dense Hiddent layer
    tf.keras.layers.Dense(64, activation="relu"),

    #output layer
    tf.keras.layers.Dense(1, activation="sigmoid")
])

cnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 300, 300, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 298, 298, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,993 (394.50 KB)

 Trainable params: 100,993 (394.50 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
cnn_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Recall(name="recall")]
)